In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\ASUS\Desktop\RAGUdemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Initialize the model
model=SentenceTransformer('all-MiniLM-L6-v2')


#sample text 
text="""
LangChain is a framework for building applications with LLMs.
Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.
You can create chains, agents, memory, and retrievers.
The Eiffel Tower is located in Paris.
France is a popular tourist destination.
"""

# step-1:split into sentences
sentences=[i.strip() for i in text.split("\n") if i.strip() ]

# step-2: embed each sentences
embeddings=model.encode(sentences)

# initialize threshold
threshold=0.75
chunks=[]
current_chunk=[sentences[0]]

# step-4: semantic grouping based on threshold
for i in range(1,len(embeddings)):
    sim=cosine_similarity(
        [embeddings[i-1]],
        [embeddings[i]]
    )[0][0]

    if sim>=threshold:
        current_chunk.append(sentences[i])
    else:
        chunks.append(" ".join(current_chunk))
        current_chunk=[sentences[i]]
    
# append the last chunk
chunks.append(" ".join(current_chunk))

# print the chunks
print("\n Semantic chunking\n")
for idx,chunk in enumerate(chunks):
    print(f"\nchunk{idx+1}:\n {chunk}")


 Semantic chunking


chunk1:
 LangChain is a framework for building applications with LLMs. Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.

chunk2:
 You can create chains, agents, memory, and retrievers.

chunk3:
 The Eiffel Tower is located in Paris.

chunk4:
 France is a popular tourist destination.


### RAG Pipeline Modular Coding

In [3]:
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
from langchain.chat_models import init_chat_model
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableMap
from langchain.prompts import PromptTemplate
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [5]:
# Create a class for semantic chunking
class ThresholdSemanticChunker:
    def __init__(self,model="all-MiniLM-L6-v2",threshold=0.7):
        self.model=SentenceTransformer(model)
        self.threshold=threshold

    def split(self,text:str):
        sentences=[s.strip() for s in text.split("\n") if s.strip()]

        # convert each sentence into vector
        embeddings=self.model.encode(sentences)

        chunks=[]
        current_chunk=[sentences[0]]
        for i in range(1,len(sentences)):
            sim=cosine_similarity([embeddings[i-1]],[embeddings[i]])[0][0]

            if sim>=self.threshold:
                current_chunk.append(sentences[i])
            else:
                chunks.append(".".join(current_chunk)+".")
                current_chunk=[sentences[i]]
        chunks.append(".".join(current_chunk)+".")
        return chunks
    
    def split_document(self,docs):
        result=[]
        for doc in docs:
            for chunk in self.split(doc.page_content):
                result.append(Document(page_content=chunk,metadata=doc.metadata))
        return result




In [6]:
# Sample text
sample_text = """
LangChain is a framework for building applications with LLMs.
Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.
You can create chains, agents, memory, and retrievers.
The Eiffel Tower is located in Paris.
France is a popular tourist destination.
"""

doc=Document(page_content=sample_text)
doc

Document(metadata={}, page_content='\nLangChain is a framework for building applications with LLMs.\nLangchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.\nYou can create chains, agents, memory, and retrievers.\nThe Eiffel Tower is located in Paris.\nFrance is a popular tourist destination.\n')

In [7]:
#chunking
chunker=ThresholdSemanticChunker()
chunks=chunker.split_document([doc])
chunks

[Document(metadata={}, page_content='LangChain is a framework for building applications with LLMs..Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone..'),
 Document(metadata={}, page_content='You can create chains, agents, memory, and retrievers..'),
 Document(metadata={}, page_content='The Eiffel Tower is located in Paris..'),
 Document(metadata={}, page_content='France is a popular tourist destination..')]

In [8]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore=FAISS.from_documents(chunks,embedding)
retriever=vectorstore.as_retriever()

In [9]:
#prompt template
template="""
    answer the question based only on the following context:{context}
    Question:{question}

    """
prompt=PromptTemplate.from_template(template)

In [10]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    answer the question based only on the following context:{context}\n    Question:{question}\n\n    ')

In [11]:
# initialize the llm
llm=init_chat_model("openai:gpt-3.5-turbo",temperature=0.4)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001940E522510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001940E522A50>, root_client=<openai.OpenAI object at 0x000001940E570E10>, root_async_client=<openai.AsyncOpenAI object at 0x000001940E5716D0>, temperature=0.4, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [12]:
#LCEL Chain with retrieval
rag_chain=(
    RunnableMap(
        {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"],  
        }
    )
    |prompt
    |llm
    |StrOutputParser()
)
rag_chain

{
  context: RunnableLambda(...),
  question: RunnableLambda(...)
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    answer the question based only on the following context:{context}\n    Question:{question}\n\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001940E522510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001940E522A50>, root_client=<openai.OpenAI object at 0x000001940E570E10>, root_async_client=<openai.AsyncOpenAI object at 0x000001940E5716D0>, temperature=0.4, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser()

In [13]:
query = {"question": "What is LangChain used for?"}
result = rag_chain.invoke(query)

print(result)

LangChain is used as a framework for building applications with LLMs and provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.


### Semantic chunker With Langchain

In [14]:
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

In [18]:
# load text
loader=TextLoader("langchain_intro.txt")
docs=loader.load()

# initialize embedding model
embeddings=OpenAIEmbeddings()

# Semantic chunker
chunker=SemanticChunker(embeddings)

# split the chunks
chunks=chunker.split_documents(docs)

for i,chunk in enumerate(chunks):
    print(f"\nchunk{i+1}:\n{chunk.page_content}")


chunk1:
LangChain is a framework for building applications with LLMs. Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.

chunk2:
You can create chains, agents, memory, and retrievers. The Eiffel Tower is located in Paris. France is a popular tourist destination.


In [20]:
db=FAISS.from_documents(chunks,embeddings)
retriever=db.as_retriever()

In [21]:
template="""
    answer the questions given context only
    {context}
    Question:{question}
    """
prompt=PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    answer the questions given context only\n    {context}\n    Question:{question}\n    ')

In [22]:
# initialize llm
llm=init_chat_model("openai:gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019418672990>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019418672350>, root_client=<openai.OpenAI object at 0x0000019419D05D90>, root_async_client=<openai.AsyncOpenAI object at 0x0000019419D05910>, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [25]:
rag_chain2=(
    RunnableMap(
        {
            "context":lambda x: retriever.invoke(x['question']),
            "question": lambda x: x["question"]
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain2

{
  context: RunnableLambda(...),
  question: RunnableLambda(...)
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    answer the questions given context only\n    {context}\n    Question:{question}\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019418672990>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019418672350>, root_client=<openai.OpenAI object at 0x0000019419D05D90>, root_async_client=<openai.AsyncOpenAI object at 0x0000019419D05910>, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser()

In [26]:
query = {"question": "What is LangChain used for?"}
result = rag_chain2.invoke(query)

print(result)

LangChain is used for building applications with LLMs and combining them with tools like OpenAI and Pinecone.
